# 1. Procesado de datos

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train.csv')

In [2]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [3]:
en_stopwords = nlp.Defaults.stop_words

def remove_stop_words(text):
    cleanText = ''
    phrase = nlp(text)
    for token in phrase:
        if not token.is_stop and not token.is_punct and not token.like_url:
            cleanText += ' ' + token.text

    return cleanText

df['text_cleaned'] = df['text'].apply(remove_stop_words)

In [4]:
df['text_cleaned']

0        Hi Roy hope ok Trans people gay thing s ramme...
1                                     fuckin hell biology
2                                  nice looking clergyman
3                           AIDS WAY SIN CONSEQUENCES BAD
4                                                   learn
                              ...                        
8143     Yeah alive time election happen fairly soon U...
8144                                  fundamentally wrong
8145     confused homosexuality big deal proud normal ...
8146                                           disgusting
8147     Peter Sørensen note Peter poor maths 13 27 eq...
Name: text_cleaned, Length: 8148, dtype: object

Separamos el conjunto en entrenamiento y test.

In [5]:
# spling training and testing
from sklearn.model_selection import train_test_split

X = df['text_cleaned']
y = df['label']

#train
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

# Define el pipeline
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

# Define los parámetros para la búsqueda de cuadrícula de TfidfVectorizer
parameters_tfidf = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__token_pattern': (r"\b\w\w+\b", r"\b[a-zA-Z]{3,}\b"),
    'tfidf__norm': ('l1', 'l2', None),
    'tfidf__use_idf': (True, False),
    'tfidf__sublinear_tf': (True, False)
}

# Realiza la búsqueda de cuadrícula para TfidfVectorizer
grid_search_tfidf = GridSearchCV(text_clf, parameters_tfidf, cv=5, n_jobs=-1)
grid_search_tfidf.fit(X_train, y_train)

# Obtén los mejores parámetros encontrados para TfidfVectorizer
best_params_tfidf = grid_search_tfidf.best_params_

# Define los parámetros para la búsqueda de cuadrícula de LinearSVC
parameters_svc = {
    'clf__penalty': ('l1', 'l2'),
    'clf__loss': ('hinge', 'squared_hinge'),
    'clf__dual': (True, False),
    'clf__tol': (1e-4, 1e-3),
    'clf__C': (0.1, 1, 10),
    'clf__multi_class': ('ovr', 'crammer_singer'),
    'clf__fit_intercept': (True, False),
    'clf__intercept_scaling': (1.0, 2.0),
    'clf__class_weight': (None, 'balanced'),
    'clf__verbose': (0, 1),
    'clf__random_state': (None, 42),
    'clf__max_iter': (1000, 2000)
}

# Actualiza el pipeline con los mejores parámetros de TfidfVectorizer
text_clf.set_params(**best_params_tfidf)

# Realiza la búsqueda de cuadrícula para LinearSVC utilizando los mejores parámetros de TfidfVectorizer
grid_search_svc = GridSearchCV(text_clf, parameters_svc, cv=5, n_jobs=-1)
grid_search_svc.fit(X_train, y_train)

# Muestra los mejores parámetros encontrados para LinearSVC
print("Mejores parámetros para LinearSVC:")
print(grid_search_svc.best_params_)

/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/pablo/anaconda3/lib/py

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]..[LibLinear].........*.*
optimization finished, #iter = 84
Objective value = -297.462880
nSV = 3984
...**
optimization finished, #iter = 73
Objective value = -298.710916
nSV = 3965
[LibLinear].......**
optimization finished, #iter = 79
Objective value = -299.394523
nSV = 3973
[LibLinear]....*
optimization finished, #iter = 42
Objective value = -298.223058
nSV = 3940
[LibLinear].....*
optimization finished, #iter = 56
Objective value = -298.634407
nSV = 3978
[LibLinear]....*
optimization finished, #iter = 48
Objective value = -297.462879
nSV = 3980
[LibLinear]....*
optimization finished, #iter = 43
Objective value = -299.394521
nSV = 3972
[LibLinear].....*
optimization finished, #iter = 52
Objective value = -298.710915
nSV = 3968
[LibLinear]...*
optimization 

/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LibLinear]...*.**.
optimization finished, #iter = 50
Objective value = 3037.286853
#nonzeros/#features = 1292/52194
[LibLinear]....................................................................................................
optimization finished, #iter = 1000

Objective value = 3078.477630
#nonzeros/#features = 1213/52217


/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

....*...............*...........................[LibLinear].................*..........................................*...........................................................*......................................*........*..................*................*..*..[LibLinear].*.
optimization finished, #iter = 1172
Objective value = 7267.651140
#nonzeros/#features = 4468/53441
........................*..............*..
optimization finished, #iter = 1284
Objective value = 7350.119126
#nonzeros/#features = 4407/52686
.......................................*.....*.....[LibLinear]..............................................................................*..............................*......................................................*.....*....................................*...................................................*.....
optimization finished, #iter = 2000

Objective value = 7391.926511
#nonzeros/#features = 4068/53010
.................*..*.*

/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


.
optimization finished, #iter = 1260
Objective value = 7410.796617
#nonzeros/#features = 4646/52194
...............*..**
optimization finished, #iter = 1189
Objective value = 7431.044825
#nonzeros/#features = 4730/52217
[LibLinear].............*..........*.......*..*
optimization finished, #iter = 322
Objective value = 7268.353456
#nonzeros/#features = 4238/53441
[LibLinear].....[LibLinear].............*..................*........*........[LibLinear]..........[LibLinear]**..........*....*
optimization finished, #iter = 363
Objective value = 7350.667656
#nonzeros/#features = 4389/52686
................*.*...........*...*
optimization finished, #iter = 406
Objective value = 7392.410220
#nonzeros/#features = 4467/53010
....*.........*.............*.....*.......*.**
optimization finished, #iter = 366
Objective value = 7411.449346
#nonzeros/#features = 4571/52194
.*
optimization finished, #iter = 359
Objective value = 7431.664160
#nonzeros/#features = 4480/52217
[LibLinear]iter  1 act 3.96

/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
7680 fits failed out of a total of 30720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1920 fits failed with the following error:
Traceback (most recent call last):
  File "/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pablo/anaconda3/lib/python3.11/site-packages/sklearn/pipeline.py", line 42

Mejores parámetros para LinearSVC:
{'clf__C': 1, 'clf__class_weight': None, 'clf__dual': True, 'clf__fit_intercept': True, 'clf__intercept_scaling': 2.0, 'clf__loss': 'hinge', 'clf__max_iter': 1000, 'clf__multi_class': 'ovr', 'clf__penalty': 'l2', 'clf__random_state': None, 'clf__tol': 0.0001, 'clf__verbose': 0}


In [10]:
best_params_svc = grid_search_svc.best_params_

# Crea una nueva pipeline con los mejores parámetros encontrados
best_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

best_pipeline.set_params(**best_params_tfidf)
best_pipeline.set_params(**best_params_svc)

best_pipeline.fit(X_train, y_train)

predictions = best_pipeline.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
print(confusion_matrix(y_test, predictions))

[[1457  243]
 [ 453  536]]


In [14]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.76      0.86      0.81      1700
           1       0.69      0.54      0.61       989

    accuracy                           0.74      2689
   macro avg       0.73      0.70      0.71      2689
weighted avg       0.74      0.74      0.73      2689



In [15]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions)

0.7411677203421346

In [16]:
best_pipeline.predict(["Kill them where they lie ...kids as well"])

NotFittedError: The TF-IDF vectorizer is not fitted